In [1]:
import numpy as np
import scipy as sc

import matplotlib.pyplot as plt
import seaborn as sns

import flowio
import flowutils

In [2]:
fd = flowio.FlowData("/media/clint/Data/GitRepo/BIOS822_FinalProj/data_test/100715.fcs")
fd

100715.fcs

# get raw data matrix

In [7]:
events = np.reshape(fd.events, (-1, fd.channel_count))
print(events.shape)
print("=================")
print(events)

(65016, 16)
[[27700.75       27291.75         177.52584839 ...  2550.91430664
   1862.98425293  1972.48535156]
 [41264.25       39764.25         320.12295532 ...  3401.5065918
   2471.22363281  2268.83178711]
 [65054.75       57606.25         203.0160675  ...  1156.66027832
    879.63946533   802.78210449]
 ...
 [29046.25       28205.75         250.55567932 ...  2121.32495117
   1562.25927734  1683.63220215]
 [48488.25       48893.           342.35766602 ...  2775.72119141
   2028.36791992  2538.3190918 ]
 [36386.         35730.           149.3036499  ...  1527.77282715
   1305.98071289  1368.90368652]]


# get spill matrix

In [4]:
spill, markers = flowutils.compensate.get_spill(fd.text['spill'])
print(spill.shape)

(13, 13)


# compensation

In [8]:
fluoro_indices = []
for channel in fd.channels:
    if fd.channels[channel]['PnN'] in markers:
        fluoro_indices.append(int(channel) - 1)

fluoro_indices.sort()
comp_events = flowutils.compensate.compensate(
    events,
    spill,
    fluoro_indices)

# arcsinh transformation

In [9]:
fluoro_indices = []
for channel in fd.channels:
    if fd.channels[channel]['PnN'] in markers:
        fluoro_indices.append(int(channel) - 1)
xform_events = flowutils.transforms.asinh(comp_events, fluoro_indices, 1)

# Question:

However, when compensation, I found some negative values.

In [19]:
print(events[0:6, 0:8])

[[27700.75       27291.75         177.52584839  1984.48461914
    625.07958984  1232.10083008   748.5100708   1553.02954102]
 [41264.25       39764.25         320.12295532  3639.62036133
    539.70324707  1433.31115723  1470.26586914  2217.67504883]
 [65054.75       57606.25         203.0160675   2191.86132812
    198.65412903   726.97979736   766.21984863   802.25213623]
 [30584.         31664.5          130.68690491  1873.40930176
   1304.08947754  2528.70825195   784.69799805  1702.36706543]
 [39505.75       39626.           203.25166321  2540.62011719
    323.26254272   857.15246582   715.00042725  1117.47753906]
 [33171.5        34794.           333.64245605  2192.86425781
   1408.85632324  2573.50952148  1604.22363281  2128.17480469]]


In [18]:
print(comp_events[0:6, 0:8])

[[ 2.77007500e+04  2.72917500e+04  1.77525848e+02  1.97993412e+03
   1.14513296e+02  8.28022177e+02  2.90623945e+02  1.05002421e+03]
 [ 4.12642500e+04  3.97642500e+04  3.20122955e+02  3.63527299e+03
  -1.00759564e+02  7.87444095e+02  8.80940009e+02  1.71133705e+03]
 [ 6.50547500e+04  5.76062500e+04  2.03016068e+02  2.19053887e+03
  -9.79089165e+01  4.06673574e+02  5.96256882e+02  6.46892469e+02]
 [ 3.05840000e+04  3.16645000e+04  1.30686905e+02  1.86496819e+03
   3.95686899e+02  2.00423786e+03  1.97533607e+02  7.81215050e+02]
 [ 3.95057500e+04  3.96260000e+04  2.03251663e+02  2.53770705e+03
  -2.26665959e+01  5.39156198e+02  3.00774329e+02  6.93336929e+02]
 [ 3.31715000e+04  3.47940000e+04  3.33642456e+02  2.18053995e+03
   4.81685199e+02  1.71846859e+03  1.07455383e+03  9.34122677e+02]]


In [20]:
print(xform_events[0:6, 0:8])

[[ 2.77007500e+04  2.72917500e+04  1.77525848e+02  8.28396609e+00
   5.43385719e+00  7.41218748e+00  6.36518029e+00  7.64971591e+00]
 [ 4.12642500e+04  3.97642500e+04  3.20122955e+02  8.89158669e+00
  -5.30590893e+00  7.36193996e+00  7.47413703e+00  8.13817751e+00]
 [ 6.50547500e+04  5.76062500e+04  2.03016068e+02  8.38505008e+00
  -5.27721088e+00  6.70115953e+00  7.08381947e+00  7.16532786e+00]
 [ 3.05840000e+04  3.16645000e+04  1.30686905e+02  8.22414653e+00
   6.67377202e+00  8.29616639e+00  5.97906232e+00  7.35399805e+00]
 [ 3.95057500e+04  3.96260000e+04  2.03251663e+02  8.53216344e+00
  -3.81452572e+00  6.98315336e+00  6.39951019e+00  7.23466377e+00]
 [ 3.31715000e+04  3.47940000e+04  3.33642456e+02  8.38047504e+00
   6.87043904e+00  8.14233609e+00  7.67280821e+00  7.53275524e+00]]
